In [1]:
import numpy as np
import pandas as pd
import os
import string


In [2]:
import gzip
from collections import defaultdict

def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        yield l.strip().split(',')


In [3]:
bookCount = defaultdict(int)
userCount = defaultdict(int)
userbook = defaultdict(set)
bookuser = defaultdict(set)
totalRead = 0
val_user = []
val_read = []
train_user = []
train_read = []
for user,book,_ in readCSV("train_Interactions.csv.gz"):
    totalRead += 1
    if totalRead >190000:
        val_user.append(user)
        val_read.append(book)
    else:
        userCount[user] +=1
        bookCount[book] += 1
        userbook[user].add(book)
        bookuser[book].add(user)
        train_user.append(user)
        train_read.append(book)

In [79]:
mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()
popularity = set()
count = 0
for ic, i in mostPopular:
    count += ic
    popularity.add(i)
    if count > totalRead/1.7: break

In [5]:
threshold = 100
booklover = [x for x in userCount if userCount[x]>threshold]

In [19]:
def Jaccard(s1,s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer/denom

In [7]:
train_unread = []
for u in train_user:
    posi = userbook[u]
    neg = set(bookCount.keys()).difference(posi)
    train_unread.append(np.random.choice(list(neg)))

In [8]:
val_unread = []
for u in val_user:
    posi = userbook[u]
    neg = set(bookCount.keys()).difference(posi)
    val_unread.append(np.random.choice(list(neg)))

In [80]:
def popular(book):
    if book in popularity:
        return 1
    else:
        return 0

In [12]:
def active_user(user):
#     if user in booklover:
#         return 1
#     else:
#         return 0
    return len(userbook[user])

In [13]:
def Jaccard_simi(user, book):
    s = []
    for b in userbook[user]:
        s.append(Jaccard(bookuser[b],bookuser[book]))
        s.sort()
        s.reverse()
    if s[0]>0.03:
        return 1
    else:
        return 0

In [22]:
def diverse_reader(user):
    books = list(userbook[user])
    s = []
    l = len(books)
    for i in range(l-1):
        for j in range(i,l,1):
            s.append(Jaccard(bookuser[books[i]],bookuser[books[j]]))
    s.sort()
    s.reverse()
    return np.mean(s)

In [23]:
diverse = defaultdict(float)
for u in train_user:
    diverse[u] = diverse_reader(u)

C:\Users\Claire\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Claire\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [25]:
len(diverse)

11357

In [82]:
X = []
y = []
for u,b in zip(train_user,train_read):
    p = popular(b)
    a = active_user(u)
    J = Jaccard_simi(u,b)
    d = diverse[u]
    feat = [p,a,J,d]
    X.append(feat)
    y.append(1)

In [83]:
for u,b in zip(train_user,train_unread):
    p = popular(b)
    a = active_user(u)
    J = Jaccard_simi(u,b)
    d = diverse[u]
    feat = [p,a,J,d]
    X.append(feat)
    y.append(0)

In [106]:
x0 = pd.DataFrame(X).fillna(0).values

In [107]:
from sklearn.linear_model import LogisticRegression
lg  = LogisticRegression().fit(x0,y)

C:\Users\Claire\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [108]:
X_val = []
y_val = []
for u,b in zip(val_user,val_read):
    p = popular(b)
    a = active_user(u)
    J = Jaccard_simi(u,b)
    d = diverse[u]
    feat = [p,a,J,d]
    X_val.append(feat)
    y_val.append(1)

In [110]:
for u,b in zip(val_user,val_unread):
    p = popular(b)
    a = active_user(u)
    J = Jaccard_simi(u,b)
    d = diverse[u]
    feat = [p,a,J,d]
    X_val.append(feat)
    y_val.append(0)

In [112]:
xv = pd.DataFrame(X_val).fillna(0).values

In [126]:
from sklearn import svm
clf = svm.SVC(gamma='scale')
clf.fit(x0, y) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [ ]:
import pandas as pd
df = pd.DataFrame(index = )

In [146]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(x0, y) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [136]:
neigh.predict(x0)

array([1, 1, 1, ..., 0, 0, 0])

In [147]:
neigh.score(xv,y_val)

0.5

In [3]:
userbook = defaultdict(set)
bookuser = defaultdict(set)
totalRead = 0
bookCount = defaultdict(int)
train_user = set()
train_book = set()
for user,book,_ in readCSV("train_Interactions.csv.gz"):
    totalRead += 1
    bookCount[book]+=1
    userbook[user].add(book)
    bookuser[book].add(user)
    train_user.add(user)
    train_book.add(book)

In [4]:
df = pd.DataFrame(index = train_user)
for i in train_book:
    u = bookuser[i]
    r = [1 if x in u else 0 for x in train_user]
    df[i] = r
df.head()

,b20017661,b08859581,b64523826,b08176289,b63619427,b31775906,b66569501,b71134815,b01673006,b69711484,...,b51723116,b26102890,b99994927,b36538437,b94507782,b34256560,b91057787,b39273891,b78149644,b28940989
u94552123,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
u17723608,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
u43906778,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
u31251874,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
u34322850,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
import math
def cosine(a,b):
    return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))

In [26]:
mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()
popularity = set()
count = 0
for ic, i in mostPopular:
    count += ic
    popularity.add(i)
    if count > totalRead/1.7: break

In [7]:
# appear = defaultdict(int)

# predictions = open("predictions_Read.txt", 'w')
# for l in open("pairs_Read.txt"):
#     if l.startswith("userID"):
#       #header
#         predictions.write(l)
#         continue
#     u,b = l.strip().split('-')
#     users = df[b]
#     cos = []
#     for book in userbook[u]:
#         cos.append(cosine(users,df[book]))
#     meancos = np.mean(cos)
#     pred = 0
#     appear[u]+=1
#     if meancos>0.005 or b in popularity:
#         predictions.write(u + '-' + b + ",1\n")
#     else:
#         predictions.write(u + '-' + b + ",0\n")
# predictions.close()


In [8]:
utest = defaultdict(set)
for l in open("pairs_Read.txt"):
    if l.startswith("userID"):
      #header
        continue
    u,b = l.strip().split('-')
    utest[u].add(b)

In [28]:
coef = 0.0045
predictions = open("predictions_Read.txt", 'w')
predictions.write('userID-bookID,prediction\n')
for p in utest:
    if p not in train_user:
        pred_book = utest[p]
        rank = [(bookCount[b],b) for b in pred_book]
        rank.sort()
        rank.reverse()
        c = 0
        for s,b in rank:
            if c<len(rank)/2:
                predictions.write(p + '-' + b + ",1\n")
            else:
                predictions.write(p + '-' + b + ",0\n")
            c+=1
        continue
    
    have_read = userbook[p]
    pred_book = utest[p]
    cosine_simi = []
    pop = []
    for pd in pred_book:
        cos = []
        for hr in have_read:
            cos.append(cosine(df[pd],df[hr]))
        cosine_simi.append(np.mean(cos))
        if pd in popularity:
            pop.append(1)
        else:
            pop.append(0)
    if p in diverse:
        score = 0.9*np.array(cosine_simi) + coef*np.array(pop)
    else:
        score = np.array(cosine_simi) + coef*np.array(pop)
    rank = list(zip(score,pred_book))
    rank.sort()
    rank.reverse()
    c = 0
    for s,b in rank:
        if c<len(rank)/2:
            predictions.write(p + '-' + b + ",1\n")
        else:
            predictions.write(p + '-' + b + ",0\n")
        c+=1

In [200]:
def new_book(booklist):
    if book in train_book:
        return Ture
    e

{'b43582370', 'b45829752'}

In [ ]:
predictions = open("predictions_Read.txt", 'w')
for l in open("pairs_Read.txt"):
    if l.startswith("userID"):
      #header
        predictions.write(l)
        continue
    u,b = l.strip().split('-')
    book = t_user[u]
    ja_simi = []
    for bs in book:
        s = Jaccard(t_book[bs],t_book[b])
        ja_simi.append(s)
    ja_simi.sort(reverse = True)
    if ja_simi[0]> 0.03:
        predictions.write(u + '-' + b + ",1\n")
    else:
        if b in return4:
            predictions.write(u + '-' + b + ",1\n")
        else:
            predictions.write(u + '-' + b + ",0\n")
predictions.close()


In [48]:
#diverse readers

In [8]:
pred_read = [1 if d in return1 else 0 for d in val_read]
pred_unread = [1 if d in return1 else 0 for d in unread]

In [9]:
accuracy = (sum(pred_read) +(len(pred_unread)-sum(pred_unread)))/(len(pred_read)+ len(pred_unread))
accuracy

0.6513848615138487